# **Setup**

## Colab Environment Setup

In order to maximize our experiments we made use both of Google Colab, Kaggle and our own machines for the training of our various models.

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

In [ ]:
%cd /gdrive/MyDrive/Colab \Notebooks/

## Python Environment Setup

Import statements



In [ ]:
import tensorflow as tf
import numpy as np
import os
import random
from datetime import datetime
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix
from tensorflow.keras.preprocessing.image import ImageDataGenerator


tfk = tf.keras
tfkl = tf.keras.layers

Hyperparameters and constants

In [ ]:
# Environment parameters
model_name = "Model_MK_XXIX"
seed = 42 + 17
random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

# Model parameters
epochs = 300
patience = 30
metrics = ["categorical_accuracy"]
monitor = "val_categorical_accuracy"
regularization = 1e-5

# Data parameters
image_size = (256, 256)
color_mode = "rgb"
input_shape = (256, 256, 3)
height_shift_range = 0.3
width_shift_range = 0.3
rotation_range = 30
shear_range = 30
zoom_range = 0.3
batch_size = 64
labels = ['Apple', 'Blueberry', 'Cherry', 'Corn', 'Grape', 'Orange', 'Peach', 'Pepper', 'Potato', 'Raspberry',
          'Soybean', 'Squash', 'Strawberry', 'Tomato']


## Dataset Initialization

Unzip challenge dataset

In [ ]:
!unzip dataset.zip

In order to apply oversampling to balance the dataset we made use of the `splitfolder` python library, which also allowed us to divide the dataset into validation and training sets directly in the data directory.

In [ ]:
!pip install split-folders
import splitfolders

splitfolders.fixed('training', output="data_oversample", seed=seed, fixed=52, oversample=True)

Setting data directories

In [ ]:
dataset_dir = 'data_oversample'
training_dir = os.path.join(dataset_dir, 'train')
validation_dir = os.path.join(dataset_dir, 'val')

## Data Preprocessing

Setup for augmentation and preprocessing

In [ ]:
# Preprocessing function for the transfer learning model that will be used
def preprocessing(x):
    x = tf.keras.applications.efficientnet.preprocess_input(x)
    return x

train_data_gen = ImageDataGenerator(preprocessing_function = preprocessing,
                                    rotation_range=rotation_range,
                                    height_shift_range=height_shift_range,
                                    width_shift_range=width_shift_range,
                                    zoom_range=zoom_range,
                                    shear_range=shear_range,
                                    horizontal_flip=True,
                                    vertical_flip=True,
                                    fill_mode='constant')
valid_data_gen = ImageDataGenerator(preprocessing_function = preprocessing,
                                    rotation_range=rotation_range,
                                    height_shift_range=height_shift_range,
                                    width_shift_range=width_shift_range,
                                    zoom_range=zoom_range,
                                    shear_range=shear_range,
                                    horizontal_flip=True,
                                    vertical_flip=True,
                                    fill_mode='constant')

Creation of the data generators

In [ ]:
train_gen = train_data_gen.flow_from_directory(directory=training_dir,
                                               target_size=image_size,
                                               color_mode=color_mode,
                                               classes=None,
                                               class_mode='categorical',
                                               batch_size=batch_size,
                                               shuffle=True,
                                               seed=seed)
valid_gen = valid_data_gen.flow_from_directory(directory=validation_dir,
                                               target_size=image_size,
                                               color_mode=color_mode,
                                               classes=None,
                                               class_mode='categorical',
                                               batch_size=batch_size,
                                               shuffle=False,
                                               seed=seed)

## Callbacks and Checkpoints

Utility function used to create checkpoints and callbacks for training, taken from the exercise sessions

In [ ]:
def create_folders_and_callbacks(model_name):
    exps_dir = os.path.join('models_saved')
    if not os.path.exists(exps_dir):
        os.makedirs(exps_dir)

    now = datetime.now().strftime('%b%d_%H-%M-%S')

    exp_dir = os.path.join(exps_dir, model_name + '_' + str(now))
    if not os.path.exists(exp_dir):
        os.makedirs(exp_dir)

    callbacks = []

    # Model checkpoint
    ckpt_dir = os.path.join(exp_dir, 'ckpts')
    if not os.path.exists(ckpt_dir):
        os.makedirs(ckpt_dir)

    ckpt_callback = tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(ckpt_dir, 'cp.ckpt'),
                                                       save_weights_only=False,  
                                                       save_best_only=False)  
    callbacks.append(ckpt_callback)

    # Tensorboard
    tb_dir = os.path.join(exp_dir, 'tb_logs')
    if not os.path.exists(tb_dir):
        os.makedirs(tb_dir)
        
    tb_callback = tf.keras.callbacks.TensorBoard(log_dir=tb_dir,
                                                 profile_batch=0,
                                                 histogram_freq=1) 
    callbacks.append(tb_callback)

    # Early Stopping
    es_callback = tf.keras.callbacks.EarlyStopping(monitor=monitor, mode='max', 
                                                   patience=patience, 
                                                   restore_best_weights=True, 
                                                   verbose=1)
    callbacks.append(es_callback)

    return callbacks



# **Transfer Learning**

## Model Setup

Supernet built through transfer learning using EfficientNetB7

In [ ]:
supernet = tfk.applications.EfficientNetB7(
    include_top=False,
    weights="imagenet",
    input_shape=input_shape,
    pooling = "avg")

supernet.summary()

# Supernet locked to be used as feature extractor
supernet.trainable = False

Dense layers added to provide classification

In [ ]:
# Declare the layers
inputs = tfk.Input(shape=input_shape)

x = supernet(inputs)
x = tfkl.Dropout(0.3, seed=seed)(x)

x = tfkl.Dense(
    64, activation='relu', 
    kernel_initializer = tfk.initializers.GlorotUniform(seed), 
    kernel_regularizer=tf.keras.regularizers.l2(regularization)
    )(x)
x = tfkl.Dropout(0.3, seed=seed)(x)

outputs = tfkl.Dense(
    14, activation='softmax',
    kernel_initializer = tfk.initializers.GlorotUniform(seed)
    )(x)


# Connect input and output 
model = tfk.Model(inputs=inputs, outputs=outputs, name=model_name)

Compile the model

In [ ]:
model.compile(loss=tfk.losses.CategoricalCrossentropy(), 
              optimizer=tfk.optimizers.Adam(), 
              metrics=metrics)
model.summary()

## Model Fitting

Training

In [ ]:
callbacks = create_folders_and_callbacks(model_name=model_name)

history = model.fit(
    x=train_gen,
    epochs=epochs,
    validation_data=valid_gen,
    callbacks=callbacks,
).history

# Save best epoch model
model.save(model_name)

Plot the training

In [ ]:
plt.figure(figsize=(15,5))
plt.plot(history['loss'], alpha=.3, color='#4D61E2', linestyle='--')
plt.plot(history['val_loss'], label='Transfer Learning', alpha=.8, color='#4D61E2')
plt.legend(loc='upper left')
plt.title('Categorical Crossentropy')
plt.grid(alpha=.3)

plt.figure(figsize=(15,5))
plt.plot(history['categorical_accuracy'], alpha=.3, color='#4D61E2', linestyle='--')
plt.plot(history['val_categorical_accuracy'], label='Transfer Learning', alpha=.8, color='#4D61E2')
plt.legend(loc='upper left')
plt.title('Accuracy')
plt.grid(alpha=.3)

plt.show()

# **Fine Tuning**

## Model Setup

Re-load model after transfer learning


In [ ]:
ft_model = tfk.models.load_model(model_name)
ft_model.summary()

# Set all EfficientNet layers to trainable
ft_model.get_layer('efficientnetb7').trainable = True
for i, layer in enumerate(ft_model.get_layer('efficientnetb7').layers):
   print(i, layer.name, layer.trainable)

# Freeze all but the last layers
for i, layer in enumerate(ft_model.get_layer('efficientnetb7').layers[:795]):
    layer.trainable = False
for i, layer in enumerate(ft_model.get_layer('efficientnetb7').layers):
    print(i, layer.name, layer.trainable)
ft_model.summary()

Compile the model

In [ ]:
ft_model.compile(loss=tfk.losses.CategoricalCrossentropy(), 
                 optimizer=tfk.optimizers.Adam(learning_rate=1e-5), 
                 metrics=metrics)

## Model Fitting

Fine tuning

In [ ]:
callbacks = create_folders_and_callbacks(model_name=model_name)

ft_history = ft_model.fit(
    x = train_gen,
    epochs = epochs,
    validation_data = valid_gen,
    callbacks = callbacks
).history

# Save the fine tuned model
ft_model.save('FineTuningModel')

Plot the training

In [ ]:
plt.figure(figsize=(15,5))
plt.plot(history['loss'], alpha=.3, color='#4D61E2', linestyle='--')
plt.plot(history['val_loss'], label='Transfer Learning', alpha=.8, color='#4D61E2')
plt.plot(ft_history['loss'], alpha=.3, color='#da6961', linestyle='--')
plt.plot(ft_history['val_loss'], label='Transfer Learning', alpha=.8, color='#da6961')
plt.legend(loc='upper left')
plt.title('Categorical Crossentropy')
plt.grid(alpha=.3)

plt.figure(figsize=(15,5))
plt.plot(history['categorical_accuracy'], alpha=.3, color='#4D61E2', linestyle='--')
plt.plot(history['val_categorical_accuracy'], label='Transfer Learning', alpha=.8, color='#4D61E2')
plt.plot(ft_history['categorical_accuracy'], alpha=.3, color='#da6961', linestyle='--')
plt.plot(ft_history['val_categorical_accuracy'], label='Transfer Learning', alpha=.8, color='#da6961')
plt.legend(loc='upper left')
plt.title('Accuracy')
plt.grid(alpha=.3)

plt.show()